**Preamble**

**Colloboration Policy**. The student is to *explicitly identify* his/her collaborators in the assignment. If the student did not work with anyone, he/she should indicate `Collaborators=['none']`. If the student obtains a solution through research (e.g., on the web), acknowledge the source, but *write up the solution in HIS/HER OWN WORDS*. There will be a one mark penalty if a student fails to indicate his/her collaborators.

**There will be NO EXCEPTIONS to this grading policy.**

## Transportation Problem

If you need help on using iPython notebooks, click <a href='#help'>here</a>. 

Assignment objectives:

i. Familiarize with the PuLP syntax and use PuLP to solve transportation problems. Click <a href='#transport_help'>here</a> for a working example.

ii. Solve *unbalanced* transportation problems by (a) using PuLP directly, and (b) introducing a dummy city and using PuLP to solve the balanced version. You expect that the answers from both methods are the same.

-----

### Mortarboard Problem

Your company manufactures mortarboards and supplies to the six universities (NUS, NTU, SMU, SUTD, SIT, SUSS) in Singapore. You have have three factories in three locations: Jurong, Woodlands, Changi.

The demand, supply and transportation costs are summarized in the following table.

|         |NUS|NTU|SMU|SUTD|SIT|SUSS|Supply|
|---------|---|---|---|----|---|----|------|
|Jurong   | 14| 2 | 25| 37 | 16| 14 | 40   |
|Woodlands| 22| 21| 29| 37 | 23| 20 | 40   |
|Changi   | 28| 37| 19| 1  | 30| 26 | 40   |
|Demand   | 39| 31| 10| 1  | 5 | 13 |      |


**(a) (7 marks)** Using PuLP directly, minimize the transportation costs while meeting the demands of all universities.

<span style="color:blue">Remember to display the costs and the number of mortarboards to be transported along each route.</span>


In [11]:
import pulp

# Creates a list of all the supply nodes
Suppliers = ["Jurong","Woodlands","Changi"]

# Creates a dictionary for the number of units of supply for each supply node
supply = {"Jurong": 40,
        "Woodlands": 40,
         "Changi": 40}

# Creates a list of all demand nodes
Schools = ["NUS", "NTU", "SMU", "SUTD", "SIT","SUSS"]

# Creates a dictionary for the number of units of demand for each demand node
demand = {"NUS": 39,
        "NTU": 31,
        "SMU": 10,
        "SUTD": 1,
        "SIT": 5,
         "SUSS":13}

# Creates a list of costs of each transportation path
costs = { "Jurong" : {"NUS" : 14, "NTU" : 2, "SMU" : 25, "SUTD" : 37, "SIT" : 16 ,"SUSS":14 },
          "Woodlands" : {"NUS" : 22, "NTU" : 21, "SMU" : 29, "SUTD" : 37, "SIT" : 23 ,"SUSS":20  },
          "Changi": {"NUS": 28, "NTU": 37, "SMU" : 19, "SUTD" : 1, "SIT" : 30 , "SUSS":26 } }

# Creates the prob variable to contain the problem data
prob = pulp.LpProblem("Mortarboard Problem (unbalanced)",pulp.LpMinimize)

# Creates a list of tuples containing all the possible routes for transport
Routes = [(ss,sb) for ss in Suppliers for sb in Schools]

# A dictionary called route_vars is created to contain the referenced variables (the routes)
route_vars = pulp.LpVariable.dicts("Route",(Suppliers, Schools),lowBound=0)

# The objective function is added to prob first
prob += pulp.lpSum([route_vars[ss][sb]*costs[ss][sb] for (ss,sb) in Routes]), "Sum of Transporting Costs"

# The supply maximum constraints are added to prob for each supply node (warehouse)
for ss in Suppliers:
    prob += pulp.lpSum([route_vars[ss][sb] for sb in Schools]) <= supply[ss], "Sum of Mortarboard out of Suppliers %s"%ss

# The demand minimum constraints are added to prob for each demand node (bar)
for sb in Schools:
    prob += pulp.lpSum([route_vars[ss][sb] for ss in Suppliers]) >= demand[sb], "Sum of Mortarboard into Schools %s"%sb

print(prob)

prob.solve()

print("model status: ",pulp.LpStatus[prob.status])

print("Minimum Cost: {}".format(pulp.value(prob.objective)))

for (ss,sb) in Routes:
      print("Amount to be transported along {}: {} with a total cost of {}".format(route_vars[ss][sb],route_vars[ss][sb].varValue,
                                                                              route_vars[ss][sb].varValue*costs[ss][sb]))




Mortarboard Problem (unbalanced):
MINIMIZE
37*Route_Changi_NTU + 28*Route_Changi_NUS + 30*Route_Changi_SIT + 19*Route_Changi_SMU + 26*Route_Changi_SUSS + 1*Route_Changi_SUTD + 2*Route_Jurong_NTU + 14*Route_Jurong_NUS + 16*Route_Jurong_SIT + 25*Route_Jurong_SMU + 14*Route_Jurong_SUSS + 37*Route_Jurong_SUTD + 21*Route_Woodlands_NTU + 22*Route_Woodlands_NUS + 23*Route_Woodlands_SIT + 29*Route_Woodlands_SMU + 20*Route_Woodlands_SUSS + 37*Route_Woodlands_SUTD + 0
SUBJECT TO
Sum_of_Mortarboard_out_of_Suppliers_Jurong: Route_Jurong_NTU
 + Route_Jurong_NUS + Route_Jurong_SIT + Route_Jurong_SMU + Route_Jurong_SUSS
 + Route_Jurong_SUTD <= 40

Sum_of_Mortarboard_out_of_Suppliers_Woodlands: Route_Woodlands_NTU
 + Route_Woodlands_NUS + Route_Woodlands_SIT + Route_Woodlands_SMU
 + Route_Woodlands_SUSS + Route_Woodlands_SUTD <= 40

Sum_of_Mortarboard_out_of_Suppliers_Changi: Route_Changi_NTU
 + Route_Changi_NUS + Route_Changi_SIT + Route_Changi_SMU + Route_Changi_SUSS
 + Route_Changi_SUTD <= 40

Sum_

---
Notice that the total supply (120) exceeds total demand (99). In other words, this is an *unbalanced* transportation problem.

To create a *balanced* instance, we introduce a "dummy" university, whose demand is the excess supply (21=120-99). The costs from each factory to the dummy university is set to zero. In other words, the modified demand, supply and transportation costs are summarized in the following table.

|         |NUS|NTU|SMU|SUTD|SIT|SUSS|Dummy|Supply|
|---------|---|---|---|----|---|----|-----|------|
|Jurong   | 14| 2 | 25| 37 | 16| 14 | 0   | 40   |
|Woodlands| 22| 21| 29| 37 | 23| 20 | 0   | 40   |
|Changi   | 28| 37| 19| 1  | 30| 26 | 0   | 40   |
|Demand   | 39| 31| 10| 1  | 5 | 13 | 21  |


**(b) (7 marks)** Using PuLP to minimize the transportation costs for the modified problem.

<span style="color:blue">Remember to display the costs and the number of mortarboards to be transported along each route.


In [12]:
import pulp

# Creates a list of all the supply nodes
Suppliers = ["Jurong","Woodlands","Changi"]

# Creates a dictionary for the number of units of supply for each supply node
supply = {"Jurong": 40,
        "Woodlands": 40,
         "Changi": 40}

# Creates a list of all demand nodes
Schools = ["NUS", "NTU", "SMU", "SUTD", "SIT","SUSS","Dummy"]

# Creates a dictionary for the number of units of demand for each demand node
demand = {"NUS": 39,
        "NTU": 31,
        "SMU": 10,
        "SUTD": 1,
        "SIT": 5,
         "SUSS":13,
         "Dummy":21}

# Creates a list of costs of each transportation path
costs = { "Jurong" : {"NUS" : 14, "NTU" : 2, "SMU" : 25, "SUTD" : 37, "SIT" : 16 ,"SUSS":14,"Dummy":0 },
          "Woodlands" : {"NUS" : 22, "NTU" : 21, "SMU" : 29, "SUTD" : 37, "SIT" : 23 ,"SUSS":20, "Dummy":0 },
          "Changi": {"NUS": 28, "NTU": 37, "SMU" : 19, "SUTD" : 1, "SIT" : 30 , "SUSS":26, "Dummy":0 } }

# Creates the prob variable to contain the problem data
prob = pulp.LpProblem("Mortarboard Problem (balanced)",pulp.LpMinimize)

# Creates a list of tuples containing all the possible routes for transport
Routes = [(ss,sb) for ss in Suppliers for sb in Schools]

# A dictionary called route_vars is created to contain the referenced variables (the routes)
route_vars = pulp.LpVariable.dicts("Route",(Suppliers, Schools),lowBound=0)

# The objective function is added to prob first
prob += pulp.lpSum([route_vars[ss][sb]*costs[ss][sb] for (ss,sb) in Routes]), "Sum of Transporting Costs"

# The supply maximum constraints are added to prob for each supply node (warehouse)
for ss in Suppliers:
    prob += pulp.lpSum([route_vars[ss][sb] for sb in Schools]) <= supply[ss], "Sum of Mortarboard out of Suppliers %s"%ss

# The demand minimum constraints are added to prob for each demand node (bar)
for sb in Schools:
    prob += pulp.lpSum([route_vars[ss][sb] for ss in Suppliers]) >= demand[sb], "Sum of Mortarboard into Schools %s"%sb

print(prob)

prob.solve()

print("model status: ",pulp.LpStatus[prob.status])

print("Minimum Cost: {}".format(pulp.value(prob.objective)))

for (ss,sb) in Routes:
    print("Amount to be transported along {}: {} with a total cost of {}".format(route_vars[ss][sb],route_vars[ss][sb].varValue,
                                                                              route_vars[ss][sb].varValue*costs[ss][sb]))



Mortarboard Problem (balanced):
MINIMIZE
37*Route_Changi_NTU + 28*Route_Changi_NUS + 30*Route_Changi_SIT + 19*Route_Changi_SMU + 26*Route_Changi_SUSS + 1*Route_Changi_SUTD + 2*Route_Jurong_NTU + 14*Route_Jurong_NUS + 16*Route_Jurong_SIT + 25*Route_Jurong_SMU + 14*Route_Jurong_SUSS + 37*Route_Jurong_SUTD + 21*Route_Woodlands_NTU + 22*Route_Woodlands_NUS + 23*Route_Woodlands_SIT + 29*Route_Woodlands_SMU + 20*Route_Woodlands_SUSS + 37*Route_Woodlands_SUTD + 0
SUBJECT TO
Sum_of_Mortarboard_out_of_Suppliers_Jurong: Route_Jurong_Dummy
 + Route_Jurong_NTU + Route_Jurong_NUS + Route_Jurong_SIT + Route_Jurong_SMU
 + Route_Jurong_SUSS + Route_Jurong_SUTD <= 40

Sum_of_Mortarboard_out_of_Suppliers_Woodlands: Route_Woodlands_Dummy
 + Route_Woodlands_NTU + Route_Woodlands_NUS + Route_Woodlands_SIT
 + Route_Woodlands_SMU + Route_Woodlands_SUSS + Route_Woodlands_SUTD <= 40

Sum_of_Mortarboard_out_of_Suppliers_Changi: Route_Changi_Dummy
 + Route_Changi_NTU + Route_Changi_NUS + Route_Changi_SIT + Route

**(c) (1 mark)** In the optimal solution, what is the amount of mortarboards left in each factory?

Leftover in Jurong:  **0**

Leftover in Woodlands: **0**

Leftover in Changi: **21**

---
<a id='transport_help'></a>

## Solving Transportation Problem using PuLP (given examples)

Extracted from this
<a href='https://www.coin-or.org/PuLP/CaseStudies/a_transportation_problem.html'>documentation</a>, click the link for more details.


In [9]:
"""
The Beer Distribution Problem for the PuLP Modeller

Authors: Antony Phillips, Dr Stuart Mitchell    2007
"""

# Import PuLP modeller functions
import pulp

# Creates a list of all the supply nodes
Warehouses = ["A","B"]

# Creates a dictionary for the number of units of supply for each supply node
supply = {"A": 1000,
        "B": 4000}

# Creates a list of all demand nodes
Bars = ["1", "2", "3", "4", "5"]

# Creates a dictionary for the number of units of demand for each demand node
demand = {"1": 500,
        "2": 900,
        "3": 1800,
        "4": 200,
        "5": 700}

# Creates a list of costs of each transportation path
costs = { "A" : {"1" : 2, "2" : 4, "3" : 5, "4" : 2, "5" : 1 },
          "B" : {"1" : 3, "2" : 1, "3" : 3, "4" : 2, "5" : 3 }}

# Creates the prob variable to contain the problem data
prob = pulp.LpProblem("Beer Distribution Problem",pulp.LpMinimize)

# Creates a list of tuples containing all the possible routes for transport
Routes = [(w,b) for w in Warehouses for b in Bars]

# A dictionary called route_vars is created to contain the referenced variables (the routes)
route_vars = pulp.LpVariable.dicts("Route",(Warehouses,Bars),lowBound=0)

# The objective function is added to prob first
prob += pulp.lpSum([route_vars[w][b]*costs[w][b] for (w,b) in Routes]), "Sum of Transporting Costs"

# The supply maximum constraints are added to prob for each supply node (warehouse)
for w in Warehouses:
    prob += pulp.lpSum([route_vars[w][b] for b in Bars]) <= supply[w], "Sum of Products out of Warehouse %s"%w

# The demand minimum constraints are added to prob for each demand node (bar)
for b in Bars:
    prob += pulp.lpSum([route_vars[w][b] for w in Warehouses]) >= demand[b], "Sum of Products into Bars %s"%b

print(prob)

prob.solve()

print("model status: ",pulp.LpStatus[prob.status])

print("Minimum Cost: {}".format(pulp.value(prob.objective)))

for (w,b) in Routes:
    print("Amount to be transported along {}: {}".format(route_vars[w][b],route_vars[w][b].varValue))

Beer Distribution Problem:
MINIMIZE
2*Route_A_1 + 4*Route_A_2 + 5*Route_A_3 + 2*Route_A_4 + 1*Route_A_5 + 3*Route_B_1 + 1*Route_B_2 + 3*Route_B_3 + 2*Route_B_4 + 3*Route_B_5 + 0
SUBJECT TO
Sum_of_Products_out_of_Warehouse_A: Route_A_1 + Route_A_2 + Route_A_3
 + Route_A_4 + Route_A_5 <= 1000

Sum_of_Products_out_of_Warehouse_B: Route_B_1 + Route_B_2 + Route_B_3
 + Route_B_4 + Route_B_5 <= 4000

Sum_of_Products_into_Bars_1: Route_A_1 + Route_B_1 >= 500

Sum_of_Products_into_Bars_2: Route_A_2 + Route_B_2 >= 900

Sum_of_Products_into_Bars_3: Route_A_3 + Route_B_3 >= 1800

Sum_of_Products_into_Bars_4: Route_A_4 + Route_B_4 >= 200

Sum_of_Products_into_Bars_5: Route_A_5 + Route_B_5 >= 700

VARIABLES
Route_A_1 Continuous
Route_A_2 Continuous
Route_A_3 Continuous
Route_A_4 Continuous
Route_A_5 Continuous
Route_B_1 Continuous
Route_B_2 Continuous
Route_B_3 Continuous
Route_B_4 Continuous
Route_B_5 Continuous

model status:  Optimal
Minimum Cost: 8600.0
Amount to be transported along Route_A_1: 